In [2]:
import json
from datetime import datetime

import pandas as pd
import requests

from config import Config

In [3]:
class DistributorApi:
    def __init__(self):
        self.base_url = "http://distributor.rules.api.pro.logitravel.internal"
        self.headers = {
            "Accept": "text/plain",
            "Authorization": f"ApiKey {Config.APIKEY}",
            "Content-Type": "application/json",
        }

        self.metadata = {
            "editState": "edit_state",
            "isObsolete": "obsolete",
            "name": "name",
            "description": "description",
            "lastUser": "updated_by",
            "lastDate": "updated_on",
            "tag": "tag",
            "id": "id",
            "rrg": "range",
            "cli": "credential",
            "ps": "paying_society",
            "cp": "refundable",
            "rat": "rate",
            "prv": "provider",
            "hot": "hotel",
            "dest": "destination",
            "mrk": "market",
            "mel": "meal",
            "cid": "check_in",
            "bod": "booking_date",
            "rel": "max_release",
            "isDynamicCommission": "dynamic_commission",
            "dow": "days_of_week",
            "age": "age",
            "room": "room",
            "non": "num_of_nights",
            "hou": "hours",
            "level_mapping": {
                "t": "level",
                "l": "list",
                "f": "from",
                "u": "to",
            },
        }

    def get_stopsale_rules(self):
        endpoint = f"/api/organizations/lgt/agencies/products/hot/level_closes/"

        try:
            r = requests.get(f"{self.base_url}{endpoint}", headers=self.headers)
            r.raise_for_status()

            print("Rules retrieved successfully.")

            return r.json()

        except requests.exceptions.HTTPError as errh:
            print("Http Error:", errh)
        except requests.exceptions.ConnectionError as errc:
            print("Error Connecting:", errc)
        except requests.exceptions.Timeout as errt:
            print("Timeout Error:", errt)
        except requests.exceptions.RequestException as err:
            print("Oops: Something Else", err)

    def stopsale_rules_to_df(self, data):
        # Flatten the "lvl" field into the main object
        data = json.loads(
            json.dumps(data),
            object_hook=lambda obj: {**obj.pop("lvl", {}), **obj},
        )
        rules = data.get("rules", [])

        for rule in rules:
            # Set default "rrg" if missing
            rrg = rule.get("rrg")
            if rrg is None:
                rule["rrg"] = {"f": 0, "t": 0, "u": 0}

            # Rename keys according to metadata
            for key in list(rule.keys()):
                if key in self.metadata:
                    rule[self.metadata[key]] = rule.pop(key)

            # Handle nested dictionary for level mapping
            level_mapping = self.metadata["level_mapping"]
            for key, value in rule.items():
                if isinstance(value, dict):
                    for k in list(value.keys()):
                        if k in level_mapping:
                            value[level_mapping[k]] = value.pop(k)

        return pd.json_normalize(rules, sep="_")

    def delete_rules(self, rule_ids):
        endpoint = "/api/organizations/lgt/agencies/products/hot/level_closes/"

        if rule_ids:
            try:
                r = requests.get(
                    f"{self.base_url}{endpoint}",
                    headers=self.headers,
                    data=json.dumps(rule_ids),
                )
                r.raise_for_status()

                print("Rules deleted successfully.")

                return r.json()

            except requests.exceptions.HTTPError as errh:
                print("Http Error:", errh)
            except requests.exceptions.ConnectionError as errc:
                print("Error Connecting:", errc)
            except requests.exceptions.Timeout as errt:
                print("Timeout Error:", errt)
            except requests.exceptions.RequestException as err:
                print("Oops: Something Else", err)
        else:
            print("No rule IDs provided.")
            return None

    def get_agencies(self):
        endpoint = "/api/organizations/lgt/agencies/"

        try:
            response = requests.get(f"{self.base_url}{endpoint}", headers=self.headers)

            if response.status_code == 200:
                return response.json()
            elif response.status_code == 404:
                print("Error: Agencies not found.")
                return None
            else:
                print(f"Error: Received unexpected status code {response.status_code}.")
                return None

        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return None

In [4]:
api = DistributorApi()

In [5]:
base_url = "http://distributor.rules.api.pro.logitravel.internal"
headers = {
    "Accept": "text/plain",
    "Authorization": f"ApiKey {Config.APIKEY}",
    "Content-Type": "application/json",
}
endpoint = "/api/organizations/lgt/agencies/products/hot/getClientGroupList"

In [21]:
response = requests.get(
    "http://distributor.rules.api.pro.logitravel.internal/api/organizations/lgt/agencies/products/hot/getClientGroupList",
    headers=headers,
)

# response2 = requests.get(
#     "http://distributor.rules.api.pro.logitravel.internal/api/organizations/lgt/agencies/products/hot/rules",
#     headers=headers,
# )

In [22]:
response.__dict__

{'_content': b'',
 '_content_consumed': True,
 '_next': None,
 'status_code': 404,
 'headers': {'Server': 'nginx', 'Date': 'Wed, 20 Nov 2024 11:50:19 GMT', 'Content-Length': '0', 'Connection': 'keep-alive'},
 'raw': <urllib3.response.HTTPResponse at 0x7fbb198397e0>,
 'url': 'http://distributor.rules.api.pro.logitravel.internal/api/organizations/lgt/agencies/products/hot/getClientGroupList',
 'encoding': None,
 'history': [],
 'reason': 'Not Found',
 'cookies': <RequestsCookieJar[]>,
 'elapsed': datetime.timedelta(microseconds=708991),
 'request': <PreparedRequest [GET]>,
 'connection': <requests.adapters.HTTPAdapter at 0x7fbb166c5250>}

In [8]:
credential_group = response.json()["clientGroupList"]

In [9]:
credential_group_mapping = {
    group["id"]: group["clientCodes"] for group in credential_group
}

In [10]:
credential_group_mapping

{'1': ['11702',
  '12123',
  '21915',
  '11617',
  '11266',
  '11047',
  '5187',
  '10921',
  '8516',
  '8550',
  '6060',
  '5943',
  '6016',
  '11123',
  '11212',
  '30783',
  '11267',
  '11913',
  '12165',
  '21170',
  '21952',
  '21533',
  '12237',
  '21165',
  '9840',
  '11945',
  '5307'],
 '2': ['8118',
  '3576',
  '3693',
  '4651',
  '5815',
  '6288',
  '8102',
  '12005',
  '12236',
  '20669',
  '20776',
  '20791',
  '11901'],
 '3': ['5906',
  '5907',
  '11948',
  '5905',
  '11873',
  '12082',
  '12122',
  '21950',
  '5226',
  '5312',
  '8119',
  '8321',
  '10205',
  '5127',
  '5138',
  '11047',
  '4800',
  '5047',
  '5462',
  '5904',
  '11127',
  '11579',
  '11753',
  '11771',
  '11986',
  '11992',
  '11993',
  '11994',
  '11996',
  '11995',
  '21008',
  '21129',
  '10150',
  '30808',
  '30830',
  '30938',
  '3693',
  '3737',
  '4194',
  '4224',
  '4772',
  '4321',
  '4506',
  '4849',
  '4853',
  '8456',
  '8515',
  '8924',
  '5318',
  '11105',
  '11106',
  '11107',
  '11108',
 